# BCSG Data Engineer - Data Engineering Project

Importing the requried packages

In [470]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3

I decided on using transfermarkt to obtain the team and player valuation data. Though most of the valuation is based on the Transfermarkt Community, they have some quantitative pointers for the same.

First, I will get the data of the clubs for the 22/23 season

In [471]:
# setting the user agent to prevent blocks for scraping
header = {'User-Agent': 
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
# for the clubs values for the 22/23 season
clubs_url = r"https://www.transfermarkt.us/laliga/startseite/wettbewerb/ES1"

In [472]:
# instantiating the requests object for the web scraping
req = requests.get(url=clubs_url, headers=header)
# instantiatingt he beautifulSoup Object
bs_obj = BeautifulSoup(req.content, "html.parser")

In [473]:
# finding the table
clubs_table = bs_obj.find('table', attrs={'class':'items'})
# drilling down to body of the table
table_body = clubs_table.find('tbody')

# instantiating the list of lists for the dataframe
clubs_df = []
# finding all the rows in the table
rows = table_body.find_all('tr')
for row in rows:
    # getting the data from each column for each row
    cols = row.find_all('td')
    # removing whitespaces around the column value text
    cols = [val.text.strip() for val in cols]
    # append list of all the value text to clubs_df i.e, each row-tuple
    clubs_df.append([val for val in cols if val]) # Get rid of empty values if they exist

In [474]:
clubs_df

[['Real Madrid', '23', '27.4', '16', '$40.05m', '$921.25m'],
 ['FC Barcelona', '25', '25.4', '10', '$35.80m', '$895.07m'],
 ['Atlético de Madrid', '22', '27.8', '16', '$30.58m', '$672.65m'],
 ['Real Sociedad', '27', '24.8', '5', '$15.13m', '$408.43m'],
 ['Villarreal CF', '25', '28.0', '11', '$15.85m', '$396.22m'],
 ['Sevilla FC', '25', '28.5', '18', '$12.17m', '$304.15m'],
 ['Real Betis Balompié', '26', '28.8', '12', '$11.00m', '$286.00m'],
 ['Valencia CF', '25', '24.4', '14', '$10.44m', '$260.92m'],
 ['Athletic Bilbao', '25', '27.3', '1', '$10.42m', '$260.59m'],
 ['Getafe CF', '24', '27.3', '12', '$6.61m', '$158.73m'],
 ['Celta de Vigo', '22', '26.0', '9', '$6.44m', '$141.57m'],
 ['CA Osasuna', '24', '27.6', '5', '$5.68m', '$136.40m'],
 ['RCD Espanyol Barcelona', '21', '26.5', '5', '$4.83m', '$101.42m'],
 ['Girona FC', '26', '26.0', '9', '$3.88m', '$100.76m'],
 ['UD Almería', '28', '25.4', '13', '$3.42m', '$95.70m'],
 ['Rayo Vallecano', '27', '28.1', '10', '$3.36m', '$90.75m'],
 ['Elc

I use pandas dataframes for easier data manipulation (i.e using an `apply()` function and intuitive removal of columns) & making bulk insertions easier through the `itertuples()` function, as you will see in subsequent steps.

In [475]:
# creating pandas dataframe of clubs_df
clubs_df1 = pd.DataFrame(clubs_df)

In [476]:
# creating the column names
clubs_df1.columns = ["Name", "Squad", "Avg Squad Age",
"Foreigners", "Avg Market Value ($)",
"Total Market Value ($)"]

In [477]:
clubs_df1.head()

,Name,Squad,Avg Squad Age,Foreigners,Avg Market Value ($),Total Market Value ($)
0,Real Madrid,23,27.4,16,$40.05m,$921.25m
1,FC Barcelona,25,25.4,10,$35.80m,$895.07m
2,Atlético de Madrid,22,27.8,16,$30.58m,$672.65m
3,Real Sociedad,27,24.8,5,$15.13m,$408.43m
4,Villarreal CF,25,28.0,11,$15.85m,$396.22m


Now, we need to format the market values, to make them be numbers.
We need to remove the `$` sign, and multiply by (10^6) if "m" , or (10^3) if "Th." exists in the string

In [478]:
# function that takes in a string of market value as input, and makes it a number
def str_to_float(str):
  """function to convert market values from string to floating point

  Args:
      str (string): string literal of market value

  Returns:
      float: the floating point representation of the string literal
  """
  # removing all whitespaces around the string
  str = str.strip()
  # get the suffix
  suffix = str[-1]
  # get the number, by removing the $ and suffix
  num = str.replace("$", "")
  # multiplication by respective power of 10
  # based on suffix
  # for a million
  if suffix == "m":
    num = float(num[:len(num)-1])
    num = num * (10 ** 6)
  # for a thousand
  else:
    num = float(num[:len(num)-3])
    num = num * (10 ** 3)
  # returning the floating point number
  return num


In [479]:
# using the apply function on the dataset
clubs_df1["Avg Market Value ($)"] = clubs_df1["Avg Market Value ($)"].apply(str_to_float)
clubs_df1["Total Market Value ($)"] = clubs_df1["Total Market Value ($)"].apply(str_to_float)

In [480]:
# checking whether the other datatypes need changing
clubs_df1.dtypes

Name                       object
Squad                      object
Avg Squad Age              object
Foreigners                 object
Avg Market Value ($)      float64
Total Market Value ($)    float64
dtype: object

We need to change the `Squad`, `Avg Squad Age`, and `Foreigners` columns to be numeric.

In [481]:
# checking the datatypes of the respective columns
clubs_df1.loc[:, ["Squad", "Avg Squad Age", "Foreigners"]] = clubs_df1.loc[:,
["Squad", "Avg Squad Age", "Foreigners"]].apply(pd.to_numeric)

In [482]:
# final check before saving onto sqlite main schema
clubs_df1.head()

,Name,Squad,Avg Squad Age,Foreigners,Avg Market Value ($),Total Market Value ($)
0,Real Madrid,23,27.4,16,40050000.0,921250000.0
1,FC Barcelona,25,25.4,10,35800000.0,895070000.0
2,Atlético de Madrid,22,27.8,16,30580000.0,672650000.0
3,Real Sociedad,27,24.8,5,15130000.0,408430000.0
4,Villarreal CF,25,28.0,11,15850000.0,396220000.0


In [483]:
clubs_df1.dtypes

Name                       object
Squad                       int64
Avg Squad Age             float64
Foreigners                  int64
Avg Market Value ($)      float64
Total Market Value ($)    float64
dtype: object

Now, we will obtain the data for the players in the clubs. I will create a list of urls for the player data by club from the *`compact`* tables on the Transfermarkt website, which will get appended in one master table for the players.

In [484]:
player_tables_urls_compact = [r"https://www.transfermarkt.us/real-madrid/kader/verein/418/saison_id/2022",
r"https://www.transfermarkt.us/fc-barcelona/kader/verein/131/saison_id/2022",
r"https://www.transfermarkt.us/atletico-de-madrid/kader/verein/13/saison_id/2022",
r"https://www.transfermarkt.us/real-sociedad/kader/verein/681/saison_id/2022",
r"https://www.transfermarkt.us/villarreal-cf/kader/verein/1050/saison_id/2022",
r"https://www.transfermarkt.us/sevilla-fc/kader/verein/368/saison_id/2022",
r"https://www.transfermarkt.us/real-betis-balompie/kader/verein/150/saison_id/2022",
r"https://www.transfermarkt.us/valencia-cf/kader/verein/1049/saison_id/2022",
r"https://www.transfermarkt.us/athletic-bilbao/kader/verein/621/saison_id/2022",
r"https://www.transfermarkt.us/getafe-cf/kader/verein/3709/saison_id/2022",
r"https://www.transfermarkt.us/celta-de-vigo/kader/verein/940/saison_id/2022",
r"https://www.transfermarkt.us/ca-osasuna/kader/verein/331/saison_id/2022",
r"https://www.transfermarkt.us/rcd-espanyol-barcelona/kader/verein/714/saison_id/2022",
r"https://www.transfermarkt.us/girona-fc/kader/verein/12321/saison_id/2022",
r"https://www.transfermarkt.us/ud-almeria/kader/verein/3302/saison_id/2022",
r"https://www.transfermarkt.us/rayo-vallecano/kader/verein/367/saison_id/2022",
r"https://www.transfermarkt.us/elche-cf/kader/verein/1531/saison_id/2022",
r"https://www.transfermarkt.us/rcd-mallorca/kader/verein/237/saison_id/2022",
r"https://www.transfermarkt.us/real-valladolid-cf/kader/verein/366/saison_id/2022",
r"https://www.transfermarkt.us/cadiz-cf/kader/verein/2687/saison_id/2022"]

In [485]:
# creating the list of lists object for the players
players = []

# over all the teams' detailed table webpages
for i in range(len(player_tables_urls_compact)):
    # instantiate new request object
    req_1 = requests.get(player_tables_urls_compact[i], headers=header)
    # instantiating the new beautifulSoup Object for the team webpage
    bs_players_obj = BeautifulSoup(req_1.content, "html.parser")
    # finding the table
    player_table = bs_players_obj.find('table', attrs={'class':'items'})
    # drilling down to body of the table
    table_body = player_table.find('tbody')

    # finding all the rows in the table
    rows = table_body.find_all('tr')
    for row in rows:
        # getting the data from each column for each row
        cols = row.find_all('td')
        # removing whitespaces around the column value text
        cols = [val.text.strip() for val in cols]
        # adding the team name
        # since it is the foreign key reference to the clubs table
        cols.append(clubs_df1["Name"][i])
        # Get rid of empty values if they exist
        # NOTE: the nationality & duplicate kit number columns turn up empty
        # due to there being more recursion required to obtain the text fields.
        # Since it isn't relevant to this project it will be removed.
        # We do not need kit number either, so it will be removed as well.
        players.append([val for val in cols])

In [486]:
# getting the first two records for the players table
players[0:6]

[['1',
  'Thibaut Courtois            \n\n\n\n\n            Goalkeeper',
  '',
  'Thibaut Courtois',
  'Goalkeeper',
  '30',
  '',
  'Jun 30, 2026',
  '$66.00m',
  'Real Madrid'],
 ['', 'Thibaut Courtois', 'Real Madrid'],
 ['Goalkeeper', 'Real Madrid'],
 ['13',
  'Andriy Lunin            \n\n\n\n\n            Goalkeeper',
  '',
  'Andriy Lunin',
  'Goalkeeper',
  '23',
  '',
  'Jun 30, 2024',
  '$2.75m',
  'Real Madrid'],
 ['', 'Andriy Lunin', 'Real Madrid'],
 ['Goalkeeper', 'Real Madrid']]

Note that we need to remove the first 2 entries in each list, which means we need to delete the lists that have lesser than 3 elements *completely* from our list of lists.

In [487]:
# removing the lists of length less than 3
players = [row for row in players if len(row) > 3]
# removing the first 2 elements of the remaining lists
players = [row[2:] for row in players]

In [488]:
players

[['',
  'Thibaut Courtois',
  'Goalkeeper',
  '30',
  '',
  'Jun 30, 2026',
  '$66.00m',
  'Real Madrid'],
 ['',
  'Andriy Lunin',
  'Goalkeeper',
  '23',
  '',
  'Jun 30, 2024',
  '$2.75m',
  'Real Madrid'],
 ['',
  'Éder Militão',
  'Centre-Back',
  '24',
  '',
  'Jun 30, 2025',
  '$66.00m',
  'Real Madrid'],
 ['',
  'David Alaba',
  'Centre-Back',
  '30',
  '',
  'Jun 30, 2026',
  '$60.50m',
  'Real Madrid'],
 ['',
  'Antonio Rüdiger',
  'Centre-Back',
  '29',
  '',
  'Jun 30, 2026',
  '$44.00m',
  'Real Madrid'],
 ['',
  'Nacho Fernández',
  'Centre-Back',
  '32',
  '',
  'Jun 30, 2023',
  '$7.70m',
  'Real Madrid'],
 ['',
  'Jesús Vallejo',
  'Centre-Back',
  '25',
  '',
  'Jun 30, 2025',
  '$3.30m',
  'Real Madrid'],
 ['',
  'Ferland Mendy',
  'Left-Back',
  '27',
  '',
  'Jun 30, 2025',
  '$44.00m',
  'Real Madrid'],
 ['',
  'Daniel Carvajal',
  'Right-Back',
  '30',
  '',
  'Jun 30, 2025',
  '$19.80m',
  'Real Madrid'],
 ['',
  'Álvaro Odriozola',
  'Right-Back',
  '26',
  '',


Now, we create the pandas dataframe for the players

In [489]:
# creating the pandas dataframe for the list of lists
players_df = pd.DataFrame(players)
players_df.columns = ["Kit No.", "Name", "Position", "Age", "Nationality", "Contract End", "Market Value", "Team"]

And we will drop the columsn which are completely empty-the kit numbers and nationalities.

In [490]:
# dropping these null columns
players_df = players_df.drop(["Kit No.", "Nationality"], axis=1)

In [491]:
players_df.head()

,Name,Position,Age,Contract End,Market Value,Team
0,Thibaut Courtois,Goalkeeper,30,"Jun 30, 2026",$66.00m,Real Madrid
1,Andriy Lunin,Goalkeeper,23,"Jun 30, 2024",$2.75m,Real Madrid
2,Éder Militão,Centre-Back,24,"Jun 30, 2025",$66.00m,Real Madrid
3,David Alaba,Centre-Back,30,"Jun 30, 2026",$60.50m,Real Madrid
4,Antonio Rüdiger,Centre-Back,29,"Jun 30, 2026",$44.00m,Real Madrid


We then format the Market Values of the players from strings to floating point numbers, and switch age to be a numeric value.

In [492]:
# now, getting the market values formatted
players_df["Market Value"] = players_df["Market Value"].apply(str_to_float)
# changing the age column's datatype to numeric
players_df["Age"] = players_df["Age"].apply(pd.to_numeric)

In [493]:
# checking the datatypes
players_df.dtypes

Name             object
Position         object
Age               int64
Contract End     object
Market Value    float64
Team             object
dtype: object

In [494]:
# getting all the positions and the number of players that play in them
players_df["Position"].value_counts()

Centre-Back           87
Central Midfield      73
Centre-Forward        66
Goalkeeper            52
Right-Back            44
Left-Back             41
Left Winger           37
Right Winger          36
Defensive Midfield    32
Attacking Midfield    25
Second Striker         3
Right Midfield         1
Name: Position, dtype: int64

In [495]:
# checking whether there are any non-unique player names
pvc = players_df["Name"].value_counts()
pvc[pvc > 1]

Series([], Name: Name, dtype: int64)

Now, we will focus on creating the optimal schema structure that this solution requires.
We know that the `clubs` table has the club name as the `primary key`.

The players table can have the player `name` as a `primary key`, with `team` being a `foreign key` reference to `clubs`.

Noting that we need to obtain a more generalised notion of position, I will create an auxilliary `Position_Map` Table, containing the mappings for each position to its more `general_position`. I am making the assumption that *wingers are an attacking position*, and thus will be classified as forwards. This means that `Position` will be a `foreign key` reference to `Position_Map`'s `Position`.

We will use the `sqlite3` module on python and run a local `sqlite` database instance on memory for this project.

In [496]:
# for the player position table
# we get the unique position values
players_df_position = players_df.loc[:, ["Position"]].copy().drop_duplicates()

In [497]:
# creating the function to generate the respective general position
def get_gen_pos(pos):
  # coercing to lowercase for easier analysis
  pos = pos.lower()
  # for goalkeeper
  if pos == "goalkeeper":
    return "Goalkeepers"
  elif "back" in pos:
    return "Defenders"
  elif "midfield" in pos:
    return "Midfielders"
  else:
    return "Forwards"

In [498]:
# creating the position map dataframe
players_df_position["general_position"] = players_df_position["Position"].apply(get_gen_pos)
players_df_position = players_df_position.reset_index().drop(["index"], axis=1)

In [499]:
players_df_position

,Position,general_position
0,Goalkeeper,Goalkeepers
1,Centre-Back,Defenders
2,Left-Back,Defenders
3,Right-Back,Defenders
4,Defensive Midfield,Midfielders
5,Central Midfield,Midfielders
6,Left Winger,Forwards
7,Right Winger,Forwards
8,Centre-Forward,Forwards
9,Attacking Midfield,Midfielders


Now, we proceed to save these 3 tables to our database schema, called `footy_schema.db`.

The entity relationship is as normalised as possible, considering the nature of the data and the dependencies. So, it is optimised enough to handle the analytical side of query processing for this project.

In [531]:
# initialising a local connection for sqlite on the RAM
conn = sqlite3.connect("footy_schema.db")

# initialising the cursor object to query the database tables,
# as mentioned on the sqlite3 documentation
cur = conn.cursor()

In [532]:
# create table statement for the clubs table
cur.execute("""
CREATE TABLE Clubs
(
  Name VARCHAR(80) PRIMARY KEY NOT NULL,
  Squad INTEGER,
  `Avg Squad Age` FLOAT,
  Foreigners INTEGER,
  `Avg Market Value ($)` DOUBLE,
  `Total Market Value ($)` DOUBLE
);
""")
# create table statement for the position_mapping table
cur.execute("""
CREATE TABLE Position_Map
(
  Position VARCHAR(60) PRIMARY KEY NOT NULL,
  `General Position` VARCHAR(40)
);
""")
# create table statement for the players table
cur.execute("""
CREATE TABLE Players
(
  Name VARCHAR(80) PRIMARY KEY NOT NULL,
  Position REFERENCES Position_Map NOT NULL,
  Age INTEGER,
  `Contract End` INTEGER,
  `Market Value ($)` DOUBLE,
  Team REFERENCES Clubs NOT NULL
);
""")

In [533]:
# checking that the tables actually are created in the database schema
cur.execute("""
SELECT name from SQLITE_MASTER WHERE type = "table";
""").fetchall()
# EXISTS!

[('Clubs',), ('Position_Map',), ('Players',)]

Now, we can only insert values as a row tuple using the sqlite3 python API's insert into statements.
So, we will reformat the pandas dataframe to get a list of row tuples, and run an `executemany()` command to perform a bulk insert.

In [534]:
# creating the list of row tuples for the clubs table
clubs_list_tuples = list(clubs_df1.itertuples(index=False, name=None))

# performing bulk insert
cur.executemany("INSERT INTO Clubs VALUES(?, ?, ?, ?, ?, ?)", clubs_list_tuples)
# committing changes to database
conn.commit()

In [535]:
# creating the list of row tuples for the position_mapping table
position_tuples = list(players_df_position.itertuples(index=False, name=None))

# performing bulk insert
cur.executemany("INSERT INTO Position_Map VALUES(?, ?)", position_tuples)
# committing changes to database
conn.commit()

In [536]:
# creating the list of row tuples for the players table
players_tuples = list(players_df.itertuples(index=False, name=None))

# performing bulk insert
cur.executemany("INSERT INTO Players VALUES(?, ?, ?, ?, ?, ?)", players_tuples)
# committing changes to database
conn.commit()

Now, we can run our query to get the total market value by team and position for teams in LaLiga 2022-2023.

In [537]:
# getting the results of the query as a pandas dataframe
qry_df = pd.read_sql_query("""
SELECT P.Team, PM.`General Position` as Position,
SUM(P.`Market Value ($)`) as `Total Market Value ($)`
FROM Players P JOIN Position_Map PM on P.Position = PM.Position
GROUP BY P.Team, PM.`General Position`
""", con=conn)

In [538]:
# we have 20 teams, with 4 general positions each. So 80 rows in total 
qry_df.shape
# Validated

(80, 3)

In [539]:
qry_df

,Team,Position,Total Market Value ($)
0,Athletic Bilbao,Defenders,65890000.0
1,Athletic Bilbao,Forwards,80850000.0
2,Athletic Bilbao,Goalkeepers,33000000.0
3,Athletic Bilbao,Midfielders,80850000.0
4,Atlético de Madrid,Defenders,168300000.0
...,...,...,...
75,Valencia CF,Midfielders,95700000.0
76,Villarreal CF,Defenders,134750000.0
77,Villarreal CF,Forwards,191400000.0
78,Villarreal CF,Goalkeepers,8470000.0


I can create a new table in the database to store the results of this query using a CTE within a `CREATE TABLE` clause. I will also save the results into a CSV, upon displaying its head and tail on this notebook as a pandas dataframe.

In [540]:
# saving the results of the query into the database,
# into a table called query_results
cur.execute("""
CREATE TABLE Query_Results AS
  SELECT P.Team, PM.`General Position` as Position,
  SUM(P.`Market Value ($)`) as `Total Market Value ($)`
  FROM Players P JOIN Position_Map PM on P.Position = PM.Position
  GROUP BY P.Team, PM.`General Position`
;
""")

In [541]:
# saving the results as a CSV file for future manipulation
qry_df.to_csv("query_results.csv")

In [542]:
# to close connection, RUN ONLY THIS STATEMENT
conn.close()

## Sanity Checks for the tables in `footy_schema.db`

In [543]:
# double check whether data tables still saved in database
# initialising a local connection for sqlite on the RAM
conn = sqlite3.connect("footy_schema.db")

# initialising the cursor object to query the database tables,
# as mentioned on the sqlite3 documentation
cur = conn.cursor()

In [544]:
# checking that the tables actually are created in the database schema
cur.execute("""
SELECT name from SQLITE_MASTER WHERE type = "table";
""").fetchall()

[('Clubs',), ('Position_Map',), ('Players',), ('Query_Results',)]

In [545]:
# storing all the tables as csvs
pd.read_sql_query("SELECT * FROM Clubs", con=conn).to_csv("clubs.csv")
pd.read_sql_query("SELECT * FROM Position_Map", con=conn).to_csv("position_map.csv")
pd.read_sql_query("SELECT * FROM Players", con=conn).to_csv("players.csv")

In [514]:
cur.execute("""
SELECT * FROM CLUBS;
""").fetchall()

[('Real Madrid', 23, 27.4, 16, 40050000.0, 921250000.0),
 ('FC Barcelona', 25, 25.4, 10, 35800000.0, 895070000.0),
 ('Atlético de Madrid', 22, 27.8, 16, 30580000.0, 672650000.0),
 ('Real Sociedad', 27, 24.8, 5, 15130000.0, 408430000.0),
 ('Villarreal CF', 25, 28.0, 11, 15850000.0, 396220000.0),
 ('Sevilla FC', 25, 28.5, 18, 12170000.0, 304150000.0),
 ('Real Betis Balompié', 26, 28.8, 12, 11000000.0, 286000000.0),
 ('Valencia CF', 25, 24.4, 14, 10440000.0, 260920000.00000003),
 ('Athletic Bilbao', 25, 27.3, 1, 10420000.0, 260589999.99999997),
 ('Getafe CF', 24, 27.3, 12, 6610000.0, 158730000.0),
 ('Celta de Vigo', 22, 26.0, 9, 6440000.0, 141570000.0),
 ('CA Osasuna', 24, 27.6, 5, 5680000.0, 136400000.0),
 ('RCD Espanyol Barcelona', 21, 26.5, 5, 4830000.0, 101420000.0),
 ('Girona FC', 26, 26.0, 9, 3880000.0, 100760000.0),
 ('UD Almería', 28, 25.4, 13, 3420000.0, 95700000.0),
 ('Rayo Vallecano', 27, 28.1, 10, 3360000.0, 90750000.0),
 ('Elche CF', 25, 28.0, 9, 3180000.0, 79420000.0),
 ('RC

In [515]:
cur.execute("""
SELECT * FROM Position_Map;
""").fetchall()

[('Goalkeeper', 'Goalkeepers'),
 ('Centre-Back', 'Defenders'),
 ('Left-Back', 'Defenders'),
 ('Right-Back', 'Defenders'),
 ('Defensive Midfield', 'Midfielders'),
 ('Central Midfield', 'Midfielders'),
 ('Left Winger', 'Forwards'),
 ('Right Winger', 'Forwards'),
 ('Centre-Forward', 'Forwards'),
 ('Attacking Midfield', 'Midfielders'),
 ('Second Striker', 'Forwards'),
 ('Right Midfield', 'Midfielders')]

In [516]:
cur.execute("""
SELECT * FROM Players;
""").fetchall()

[('Thibaut Courtois',
  'Goalkeeper',
  30,
  'Jun 30, 2026',
  66000000.0,
  'Real Madrid'),
 ('Andriy Lunin', 'Goalkeeper', 23, 'Jun 30, 2024', 2750000.0, 'Real Madrid'),
 ('Éder Militão',
  'Centre-Back',
  24,
  'Jun 30, 2025',
  66000000.0,
  'Real Madrid'),
 ('David Alaba', 'Centre-Back', 30, 'Jun 30, 2026', 60500000.0, 'Real Madrid'),
 ('Antonio Rüdiger',
  'Centre-Back',
  29,
  'Jun 30, 2026',
  44000000.0,
  'Real Madrid'),
 ('Nacho Fernández',
  'Centre-Back',
  32,
  'Jun 30, 2023',
  7700000.0,
  'Real Madrid'),
 ('Jesús Vallejo',
  'Centre-Back',
  25,
  'Jun 30, 2025',
  3300000.0,
  'Real Madrid'),
 ('Ferland Mendy', 'Left-Back', 27, 'Jun 30, 2025', 44000000.0, 'Real Madrid'),
 ('Daniel Carvajal',
  'Right-Back',
  30,
  'Jun 30, 2025',
  19800000.0,
  'Real Madrid'),
 ('Álvaro Odriozola',
  'Right-Back',
  26,
  'Jun 30, 2024',
  11000000.0,
  'Real Madrid'),
 ('Aurélien Tchouameni',
  'Defensive Midfield',
  22,
  'Jun 30, 2028',
  88000000.0,
  'Real Madrid'),
 ('Fed

In [517]:
cur.execute("""
SELECT * FROM Query_Results;
""").fetchall()

[('Athletic Bilbao', 'Defenders', 65890000.0),
 ('Athletic Bilbao', 'Forwards', 80850000.0),
 ('Athletic Bilbao', 'Goalkeepers', 33000000.0),
 ('Athletic Bilbao', 'Midfielders', 80850000.0),
 ('Atlético de Madrid', 'Defenders', 168300000.0),
 ('Atlético de Madrid', 'Forwards', 264000000.0),
 ('Atlético de Madrid', 'Goalkeepers', 47850000.0),
 ('Atlético de Madrid', 'Midfielders', 192500000.0),
 ('CA Osasuna', 'Defenders', 46750000.0),
 ('CA Osasuna', 'Forwards', 49500000.0),
 ('CA Osasuna', 'Goalkeepers', 12210000.0),
 ('CA Osasuna', 'Midfielders', 27940000.0),
 ('Celta de Vigo', 'Defenders', 44000000.0),
 ('Celta de Vigo', 'Forwards', 37950000.0),
 ('Celta de Vigo', 'Goalkeepers', 5720000.0),
 ('Celta de Vigo', 'Midfielders', 53900000.0),
 ('Cádiz CF', 'Defenders', 21340000.0),
 ('Cádiz CF', 'Forwards', 23100000.0),
 ('Cádiz CF', 'Goalkeepers', 7480000.0),
 ('Cádiz CF', 'Midfielders', 19580000.0),
 ('Elche CF', 'Defenders', 21230000.0),
 ('Elche CF', 'Forwards', 38500000.0),
 ('Elche 

In [546]:
# closing connection
conn.close()